# Başlangıç

Model için gerekli modülleri kurulması

In [ ]:
!pip install scikitplot
!pip install chart_studio
!pip install sklearn
!pip install transformers
!pip install simpletransformers
!pip install tqdm

Model için gereken kütüphanelerin yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import multiprocessing
from sklearn.metrics import accuracy_score
import nltk
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec,doc2vec
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, plot_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import umap
import scikitplot as skplt
from sklearn import metrics,utils
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('turkish'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from simpletransformers.classification import ClassificationModel
import torch,sklearn
from torch import nn
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
nltk.download('stopwords')
nltk.download('punkt')

# Veri Setinin Eklenmesi ve Ön İşleme Tabi Tutulması

Veri setimizin yüklemesini gerçekleştiririz
Mecvut analizimiz için bir kolonda verinin kendisi, diğer kolonda ise verinin etiketinin bulunması yeterlidir.
text,	labels

	mağazamın ınstagram hesabı için reklam vermeyi...	4
	google şirket reklamı vermek istiyordum bu kon...	2
	ben ınstagram	4
	kolay glsin ben web siteme reklam almak istiyo...	2
	sitemin trafiğini arttırmak için reklam vermey...	0

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train[['text','labels']]
test = test[['text','labels']]

Veri Setimize Göz Atıyoruz

Eğitim Seti:

In [ ]:
import seaborn as sns
cnt_pro = train['labels'].value_counts()

plt.figure(figsize=(12,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of Questions', fontsize=12)
plt.xlabel('labels', fontsize=12)
plt.xticks(rotation=90)
plt.savefig('fname', dpi=1200, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)
plt.show();

Test Seti:

In [ ]:
import seaborn as sns
cnt_pro = test['labels'].value_counts()

plt.figure(figsize=(12,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of Questions', fontsize=12)
plt.xlabel('labels', fontsize=12)
plt.xticks(rotation=90)
plt.savefig('fname', dpi=1200, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)
plt.show();

Veri ön işleme aşamasında, veri setimizi küçük harflere dönüştürüp, noktalama işaretleri ve sembollerden arındırılmış hale getirir ve tokenize ederiz.

In [ ]:
def cleanText(input_sentence):
 
  tmp= [word.replace('I','ı') for word in input_sentence.split(' ')]
  tmp= [word.lower() for word in tmp]
  tmp= [word.replace('i̇','i') for word in tmp]
  tmp = [re.sub('[^A-Za-z0-9ğüşıçöiâî]+', ' ', word) for word in tmp]
  tmp = [word.strip(' ') for word in tmp]
  tmp1 =' '.join(tmp)
  return tmp1

#Temizleme fonksyonunun uygulanması
train['text']=train['text'].apply(cleanText)
test['text']=test['text'].apply(cleanText)


# Doc2Vec Modeli İle AutoTagging Yapılması

Doc2Vec Modeli ile veri setimiz için RandomForest, C-SVC ve Lojistik Regresyon sınıflandırma modelleri denenmiş en iyi sonuç Lojistik Regresyon modelinden alındığı için sınıflandırıcı olarak bu model seçilmiştir. Aşağıda Lojistik Regresyon modelini Doc2Vec embeddinglerine uygulamak ve sonuçları çıktı almak için yazılmış fonksyon bulunmaktadır.

In [ ]:
def LogReg_Results(train_embeddings,test_embeddings):
  logreg = LogisticRegression(n_jobs=cores, C = 10000, max_iter=10000, tol = 1e-5)
  logreg.fit(train_embeddings, tr_tags)    
  te_tags_predicted = logreg.predict(test_embeddings)
  tr_tags_predicted = logreg.predict(train_embeddings)

  print('Train accuracy %s' % accuracy_score(tr_tags, tr_tags_predicted))
  print('Testing accuracy %s' % accuracy_score(te_tags, te_tags_predicted))
  print('Testing Log-Reg F1 weighted score: {}'.format(f1_score(te_tags, te_tags_predicted, average='weighted')))
  print('Testing Log-Reg F1 macro score: {}'.format(f1_score(te_tags, te_tags_predicted, average='macro')))
  print('Testing Log-Reg F1 macro score class-based:: {}'.format(f1_score(te_tags, te_tags_predicted, average=None)))

Eğitim ve test veri setini etiketleyip tokenized hale getiriyoruz.

In [ ]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
#Veri Setinin Doc2Vec Modeline Uygun Şekilde Tokenize Edilmesi
train_tagged = train.apply(lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.labels]), axis=1)
test_tagged = test.apply(lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.labels]), axis=1)
tr_tags = [i.tags[0] for i in train_tagged]
te_tags = [i.tags[0] for i in test_tagged] 

Modeli D-BOW ve DM modelleri için farklı epochlarda ve vektör uzunluklarında eğitip sonuçlarını gözlemliyoruz.

In [ ]:
cores = multiprocessing.cpu_count()
for vs in [50,75,100,150,200,250,300]:  
  for epoch in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    for dm in [0,1]:
      print('\n','setting : ','dm : ', dm, 'vs : ', vs, 'num epochs : ', epoch)
      model_dm = Doc2Vec(dm=dm, vector_size=vs, window=10, negative=5, min_count=1, workers=cores, alpha=0.0061, min_alpha = 0.0001)
      model_dm.build_vocab([x for x in tqdm(train_tagged.values)])

      for ep in range(epoch):
        model_dm.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
        model_dm.alpha -= 0.0012
        model_dm.min_alpha = model_dm.alpha
      
      train_embeddings = [] 
      for i in train_tagged:
        embedding = model_dm.infer_vector(i.words, steps=20, alpha = 0.005)
        train_embeddings.append(embedding)        

      test_embeddings = [] 
      for i in test_tagged:
        embedding = model_dm.infer_vector(i.words, steps=20, alpha = 0.005)
        test_embeddings.append(embedding)     

      LogReg_Results(train_embeddings,test_embeddings)
      
      

# LSTM Modeli ile AutoTagging Yapılması

LSTM modeli için gereken parametreleri belirliyor ve tokenizerımızı oluşturuyoruz.

In [ ]:
# En Çok Kullanılan Kelimeler
MAX_NB_WORDS = 50000
# Modelimizin inputlarının maksimum uzunluğu.
MAX_SEQUENCE_LENGTH = 250
# Oluşacak olan Gömülerin Boyutu.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['text'].values)
tokenizer.fit_on_texts(test['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Eğitim seti verisi ve etiketleri, Test seti verisi ve etiketlerinin boyutlarını atayıp gözlemliyoruz. 

In [ ]:
X = tokenizer.texts_to_sequences(train['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

X_tes = tokenizer.texts_to_sequences(test['text'].values)
X_tes = pad_sequences(X_tes, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_tes.shape)

Derin öğrenme LSTM modelimizin katmanlarını oluşturuyoruz. Modelimizdeki etiket sayısı kadar dense katmanı parametresi değeri giriyoruz. Modelimizde 10 etiket olduğu için dense katmanımıza 10 değerini verdik.

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
print(model.summary())

Modelimizi 20 iterasyonda eğitiyor, daha sonra oluşan modeli test seti ile doğruluk kontrolüne tabi tutuyor, ürettiğimiz modeli kaydediyor ve sınıflandırma raporları için tahmin edilen değerler ile gerçek değerleri liste formatına sokuyoruz.

In [ ]:
epochs = 20
batch_size = 64
history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
accr = model.evaluate(X_tes,Y_tes,return_dict=False)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
model.save('/LSTM10label/')
y_pred=[]
pred = model.predict(X_tes)
for i in range(len(pred)):
  y_pred.append([np.argmax(pred[i])])
Y_pred = [item for sublist in y_pred for item in sublist]
y_true = []
for i in range(len(X_tes)):
  y_true.append(Y_tes[i])

Isı haritalaması yöntemi ile karmaşıklık matrisini oluşturuyoruz.

In [ ]:
data = {'y_Actual':    y_true,
        'y_Predicted': Y_pred
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])

sns.heatmap(confusion_matrix, annot=False,robust=False)
plt.show()

Nümerik değerler ile karmaşıklık matrisini oluşturuyoruz.

In [ ]:
confusion_matrix(y_true, Y_pred)

Sınıflandırma raporunu oluşturuyoruz.

In [ ]:
print(classification_report(y_true, Y_pred))

Yeni bir sorguda modelin tahminlediği cevabı gözlemlemek için aşağıdaki kodu kullanıyoruz. Ayrıca model alt parametrelerini de görüntüleyebiliyoruz.

In [ ]:
new_querry = ['googleda reklam vermek istiyorum']
new_querry=cleanText(new_querry)
seq = X_test[0] 
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
print(seq)
print(padded)
print(pred, labels[np.argmax(pred)])

# BERT-İnceAyarlama Yöntemi ile AutoTagging Yapılması

Bert için üretilmiş base uncased(küçük harf, noktalama işaretleri ve sembollerden arınmış) modeli(https://huggingface.co/loodos/bert-base-turkish-uncased) kendi veri setimiz ile ince ayar yapmak için aşağıdaki parametreleri kullanıyoruz. 3 epochta iterasyon yapıyoruz. Sınıflandırma modelini tercih ediyoruz. 

In [ ]:
model_args = {
    "use_early_stopping": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 5,
    "evaluate_during_training_steps": 1000,
    "fp16": False,
    "num_train_epochs":3
}

model = ClassificationModel(
    "bert", 
    "dbmdz/bert-base-turkish-uncased",
     use_cuda=True, 
     args=model_args, 
     num_labels=10
)
model.train_model(train, acc=sklearn.metrics.accuracy_score,output_dir='/BERT-finetune-10labels-questions/')

İnce ayar yaptığımız BERT modelini çağırıyoruz ve ürettiğimiz etiketlere göre modelin test setimizi tahminlemesi için gereken yapıyı oluşturuyoruz.

In [ ]:
code_to_label={}
for i in range(10):
  code_to_label.update({'LABEL_%s'%i: i})
tokenizer= AutoTokenizer.from_pretrained("/BERT-finetune-10labels-questions/checkpoint-3621-epoch-3")

# Ürettiğimiz modeli yüklüyoruz
model= AutoModelForSequenceClassification.from_pretrained("/BERT-finetune-10labels-questions/checkpoint-3621-epoch-3")

# Tahminleme modelini oluşturuyoruz
nlp=pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
code_to_label=code_to_label

print(nlp("reklam vermek"))
print(code_to_label[nlp("nasıl reklam verebilirim")[0]['label']])

Test setimizdeki verileri ürettiğimiz tahminleme yapısından geçirip sonuçları karşılaştırmak için raporluyoruz.

In [ ]:
all_count = 0
true_count = 0
y_pred = []
y_true = []
for i in range(len(test['text'])):
  x_pred=None
  x_pred=code_to_label[nlp(test['text'][i])[0]['label']]
  y_pred.append(x_pred)
for i in range(len(test['text'])):
  y_true.append(test['labels'][i])
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))